In [1]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
except:
    print("Mounting Failed.")


!unzip -u /content/gdrive/MyDrive/Colab\ Notebooks/DL\ Final\ Project/feathersv1-dataset-master.zip #Logan's

!unzip -u /content/gdrive/MyDrive/Colab\ Notebooks/DL\ Final\ Project/Sample\ Images.zip

Streaming output truncated to the last 5000 lines.
   creating: feathersv1-dataset-master/images/passeriformes/sitta_europaea/
  inflating: feathersv1-dataset-master/images/passeriformes/sitta_europaea/passeriformes_sitta_europaea_00000.jpg  
  inflating: feathersv1-dataset-master/images/passeriformes/sitta_europaea/passeriformes_sitta_europaea_00001.jpg  
  inflating: feathersv1-dataset-master/images/passeriformes/sitta_europaea/passeriformes_sitta_europaea_00002.jpg  
  inflating: feathersv1-dataset-master/images/passeriformes/sitta_europaea/passeriformes_sitta_europaea_00003.jpg  
  inflating: feathersv1-dataset-master/images/passeriformes/sitta_europaea/passeriformes_sitta_europaea_00004.jpg  
  inflating: feathersv1-dataset-master/images/passeriformes/sitta_europaea/passeriformes_sitta_europaea_00005.jpg  
  inflating: feathersv1-dataset-master/images/passeriformes/sitta_europaea/passeriformes_sitta_europaea_00006.jpg  
  inflating: feathersv1-dataset-master/images/passeriformes/s

In [2]:
!git clone https://huggingface.co/spaces/sameraslan/dl_final_project_finegrainedimagefeathers
!pip install gradio

Cloning into 'dl_final_project_finegrainedimagefeathers'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.
     |████████████████████████████████| 5.2 MB 15.2 MB/s 
     |████████████████████████████████| 54 kB 120 kB/s 
     |████████████████████████████████| 253 kB 74.3 MB/s 
     |████████████████████████████████| 84 kB 4.2 MB/s 
     |████████████████████████████████| 2.0 MB 64.4 MB/s 
     |████████████████████████████████| 212 kB 75.8 MB/s 
     |████████████████████████████████| 1.1 MB 59.4 MB/s 
     |████████████████████████████████| 53 kB 2.8 MB/s 
     |████████████████████████████████| 271 kB 75.3 MB/s 
     |████████████████████████████████| 144 kB 72.3 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 10.9 MB 58.0 MB/s 
     |██████████████████

In [3]:
import os

## External Libraries
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as functional
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
from skimage import io
import matplotlib.pyplot as plt
import time

from sklearn.preprocessing import LabelEncoder
from torchvision.transforms.functional import to_tensor
from sklearn.model_selection import train_test_split

data_dir = "./feathersv1-dataset-master"

DATA_SUBSET = 'top_100'

train_csv_path = f"{data_dir}/data/train_{DATA_SUBSET}_species.csv"
test_csv_path = f"{data_dir}/data/test_{DATA_SUBSET}_species.csv"

gpu_bool = torch.cuda.is_available()

def get_paths(csv_file, data_dir="./feathersv1-dataset-master/"):
    with open(csv_file, "r") as readfile:

        readfile.readline()  # skip header
        csv_data = readfile.readlines()

    image_paths = []
    for line in csv_data:
        path_parts = [x.lower().replace(" ", "_") for x in line.strip().split(",")]

        #data_dir/images/order_name/species_name/image_file_name
        image_path = (data_dir + '/images/' + path_parts[1] + '/' + path_parts[2] + '/' + path_parts[0]) 

        #append a string representation of the path
        image_paths.append(image_path)

    return image_paths

def get_labels(csv_path):
    # Return classes from CSV file.
    with open(csv_path, "r") as readfile:
        readfile.readline()  # skip header

        labels = []
        for line in readfile.readlines():
          split_line = line.strip().split(",")
          order = split_line[1]
          species = split_line[2]
          labels.append(order + '_' + species)

        le = LabelEncoder()
        le.fit(labels)
        int_labels = le.transform(labels)
        orig_labels = le.inverse_transform(int_labels)

        print(sorted(list(set(orig_labels))))
        return int_labels

class FeatherDataset(Dataset):
    def __init__(self, image_paths, labels, transforms=[to_tensor]):

        self.image_paths = image_paths
        self.transforms = transforms
        self.labels = labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):

        img_path = self.image_paths[idx]
        label = self.labels[idx]
        img = io.imread(img_path)

        for transform in self.transforms:
          img = transform(img)
        
        return img, label

train_transforms = [to_tensor, 
                    transforms.Resize((64,64)), 
                    transforms.RandomHorizontalFlip(0.5), 
                    transforms.RandomVerticalFlip(0.5),
                    transforms.ColorJitter(brightness=[0.8,1.2], contrast=[0.8,1.2], saturation=[0.8,1.2], hue=0)]

test_transforms = [to_tensor, transforms.Resize((64,64))]

X_train, X_val, y_train, y_val = train_test_split(get_paths(train_csv_path), get_labels(train_csv_path), train_size=0.8, shuffle=True, random_state=42)
train_dataset = FeatherDataset(X_train, y_train, transforms=train_transforms)

val_dataset = FeatherDataset(X_val, y_val, transforms=test_transforms)
test_dataset = FeatherDataset(get_paths(test_csv_path),  get_labels(test_csv_path), transforms=test_transforms)

train_batch_size = 32
test_batch_size = 1
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=train_batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

['Accipitriformes_Accipiter Gentilis', 'Accipitriformes_Accipiter Nisus', 'Accipitriformes_Aquila Chrysaetos', 'Accipitriformes_Aquila Nipalensis', 'Accipitriformes_Aquila Pomarina Clanga Pomarina', 'Accipitriformes_Buteo Buteo', 'Accipitriformes_Buteo Rufinus', 'Accipitriformes_Circus Aeruginosus', 'Accipitriformes_Circus Cyaneus', 'Accipitriformes_Milvus Milvus', 'Accipitriformes_Pandion Haliaetus', 'Anseriformes_Aix Galericulata', 'Anseriformes_Aix Sponsa', 'Anseriformes_Anas Acuta', 'Anseriformes_Anas Bahamensis', 'Anseriformes_Anas Clypeata', 'Anseriformes_Anas Crecca', 'Anseriformes_Anas Platyrhynchos', 'Anseriformes_Anas Querquedula', 'Anseriformes_Anser Albifrons', 'Anseriformes_Anser Anser', 'Anseriformes_Anser Brachyrhynchus', 'Anseriformes_Anser Fabalis', 'Anseriformes_Aythya Ferina', 'Anseriformes_Aythya Fuligula', 'Anseriformes_Aythya Marila', 'Anseriformes_Callonetta Leucophrys', 'Anseriformes_Chenonetta Jubata', 'Anseriformes_Melanitta Nigra', 'Anseriformes_Mergellus Alb

In [4]:
## Load Model
def loadModel(model, filename):
  model_save_name = filename
  path = F"/content/gdrive/My Drive/Colab Notebooks/DL Final Project/Models/{model_save_name}"
  trained_model = model
  trained_model.load_state_dict(torch.load(path))
  return trained_model

In [5]:
def get_written_labels(csv_path):
    # Return classes from CSV file.
    with open(csv_path, "r") as readfile:
        readfile.readline()  # skip header

        labels = []
        for line in readfile.readlines():
          split_line = line.strip().split(",")
          order = split_line[1]
          species = split_line[2]
          labels.append(order + ' ' + species)

        le = LabelEncoder()
        le.fit(labels)
        int_labels = le.transform(labels)
        orig_labels = le.inverse_transform(int_labels)
        
        return list(le.classes_)

DATA_SUBSET = 'top_100'
train_csv_path = f"{data_dir}/data/train_{DATA_SUBSET}_species.csv"
test_csv_path = f"{data_dir}/data/test_{DATA_SUBSET}_species.csv"
labels_text = get_written_labels(test_csv_path)


In [6]:
class BNN_matmul(nn.Module):
    
    """
    
    """
    def __init__(self, n_classes):

        super(BNN_matmul, self).__init__()
        
        self.n_classes = n_classes

        # Downgrade stages
        self.VGGa = vgg16(pretrained = False).features[:15]
        # for element in self.VGGa.parameters():
        #   element.requires_grad = False
        self.VGGb = vgg16(pretrained = False).features[:15]
        self.reduction = nn.Conv2d(256, 160,  kernel_size=(1, 1), stride=(1, 1), padding=(0, 0))

        # for element in self.VGGa.parameters():
        #   element.requires_grad = False

        self.classifier = vgg16().classifier

        self.classifier[0] =  nn.Linear(in_features=160*160, 
                                        out_features=4096, 
                                        bias=True)
        self.classifier[-1] = nn.Linear(in_features=4096, 
                                        out_features=self.n_classes, 
                                        bias=True)

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        """
        Forward pass
        """
        outa = self.reduction(self.VGGa(x))
        outb = self.reduction(self.VGGb(x))
        outa_reshape = outa.reshape([outa.shape[0], outa.shape[1], -1])
        outb_reshape = outb.reshape([outb.shape[0], outb.shape[1], -1])
        outb_transpose = torch.permute(outb_reshape, (0,2,1))
        combo = torch.bmm(outa_reshape, outb_transpose) / (outa_reshape.shape[2]*2) # this should be an outer product, did they mean kroenecker product ???????
        combo_reshape = combo.reshape([combo.shape[0], -1])
        x = torch.nn.functional.normalize(torch.sign(combo_reshape) * torch.sqrt(torch.abs(combo_reshape) + 1e-10))
        unormalized_scores = self.classifier(x)
        normed_scores = unormalized_scores
        return normed_scores

In [7]:
import gradio as gr
from torchvision.models import vgg16
from sklearn.utils.class_weight import compute_class_weight
from sklearn import preprocessing


def feather_classifier(image):
  model = BNN_matmul(100)
  out = len(set(y_train))

  # modify task head
  model._modules['classifier'][6] = nn.Linear(in_features=4096, 
                                              out_features=out, 
                                              bias=True)
  
  filename = "Final Models/bnn_logan_test.pt"
  model = loadModel(model, filename)
  model.eval()


  # Image preprocessing
  image = to_tensor(image)
  image = transforms.Resize((64,64))(image)
  image = image[None, :, :, :]

  outputs = model.forward(image)
  predicted = torch.argmax(outputs, dim=1)
  predicted = predicted[0].item()
  scores_6, predicted_6 = torch.topk(outputs, k=6, dim=1)
  prediction = labels_text[predicted]

  predicted_5 = predicted_6.tolist()[0][:5]
  scores_6 = scores_6.tolist()[0]

  K = scores_6[-1]
  if K < 0:
    scores_5 = [x + (-1 * K) for x in scores_6[:5]]
  else:
    scores_5 = scores_6[:5]

  normed_scores_5 = preprocessing.normalize(np.array([scores_5]))[0]
  normed_scores_5 = normed_scores_5 / np.sum(normed_scores_5)
  normed_scores_5 = normed_scores_5.tolist()
  labels_dict = dict(zip([labels_text[i] for i in predicted_5], normed_scores_5))
  return labels_dict


import os
# assign directory
directory = "Sample Images/"
allImages = []

# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)

    # checking if it is a file
    if os.path.isfile(f):
      allImages.append(f)
    

iface = gr.Interface(feather_classifier,
                     title = "Bird Feather Species Classifier - Hierarchical Bilinear Model",
                     description = "A bird feather species fined grained image classifier by Ryunosuke Saito, Logan Donaldson, Samer Aslan, and Hashem AlSabi",
                     examples = allImages,
                     inputs = gr.inputs.Image(),
                     outputs = gr.outputs.Label(num_top_classes=5),
                     capture_session=True)
iface.launch(debug=True)

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `capture_session` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://37503.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f0458177310>,
 'http://127.0.0.1:7860/',
 'https://37503.gradio.app')

In [10]:
!pip install session-info

     |████████████████████████████████| 63 kB 2.5 MB/s 
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8048 sha256=1da86adadde1b385a4269818d2faa16ad549a7ac3fe27d575350ef07985f7c93
  Stored in directory: /root/.cache/pip/wheels/bd/ad/14/6a42359351a18337a8683854cfbba99dd782271f2d1767f87f
Successfully built session-info


In [11]:
import session_info
session_info.show()